# <center><p style ="color: gold; font-weight: 500">House Prices competition</p></center>

<center>>>> Nguyễn Ngọc Trí Vĩ <<<

Link competitions: https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques

### Import packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from xgboost import XGBRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error


### Import and preprocess data

In [133]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

##### Visualization train set

In [4]:
train_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [76]:
train_df.describe().iloc[:,0:9]

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,69.863699,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.117123
std,421.610009,42.300571,22.027677,9981.264932,1.382997,1.112799,30.202904,20.645407,180.731373
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000
25%,365.750000,20.000000,60.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000
75%,1095.250000,70.000000,79.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,164.250000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000


In [42]:
train_df.describe().iloc[:,10:19]

,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534
std,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753
min,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000
25%,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000
50%,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000
75%,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000
max,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000


In [55]:
train_df.describe().iloc[:,20:29]

,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000
mean,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521
std,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794
min,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000
50%,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000
75%,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000
max,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000


In [56]:
train_df.describe().iloc[:,30:38]

,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [75]:
train_df.iloc[:,0:27].isnull().sum()

Id              0
MSSubClass      0
MSZoning        0
LotFrontage     0
LotArea         0
Street          0
Alley           0
LotShape        0
LandContour     0
Utilities       0
LotConfig       0
LandSlope       0
Neighborhood    0
Condition1      0
Condition2      0
BldgType        0
HouseStyle      0
OverallQual     0
OverallCond     0
YearBuilt       0
YearRemodAdd    0
RoofStyle       0
RoofMatl        0
Exterior1st     0
Exterior2nd     0
MasVnrType      0
MasVnrArea      0
dtype: int64

In [74]:
train_df.iloc[:,28:52].isnull().sum()

ExterCond       0
Foundation      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinSF1      0
BsmtFinType2    0
BsmtFinSF2      0
BsmtUnfSF       0
TotalBsmtSF     0
Heating         0
HeatingQC       0
CentralAir      0
Electrical      0
1stFlrSF        0
2ndFlrSF        0
LowQualFinSF    0
GrLivArea       0
BsmtFullBath    0
BsmtHalfBath    0
FullBath        0
HalfBath        0
BedroomAbvGr    0
dtype: int64

In [73]:
train_df.iloc[:,53:81].isnull().sum()

KitchenQual      0
TotRmsAbvGrd     0
Functional       0
Fireplaces       0
FireplaceQu      0
GarageType       0
GarageYrBlt      0
GarageFinish     0
GarageCars       0
GarageArea       0
GarageQual       0
GarageCond       0
PavedDrive       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
PoolQC           0
Fence            0
MiscFeature      0
MiscVal          0
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
dtype: int64

##### Visualization test set

In [34]:
test_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [35]:
test_df.describe().iloc[:,0:9]

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea
count,1459.000000,1459.000000,1232.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1444.000000
mean,2190.000000,57.378341,68.580357,9819.161069,6.078821,5.553804,1971.357779,1983.662783,100.709141
std,421.321334,42.746880,22.376841,4955.517327,1.436812,1.113740,30.390071,21.130467,177.625900
min,1461.000000,20.000000,21.000000,1470.000000,1.000000,1.000000,1879.000000,1950.000000,0.000000
25%,1825.500000,20.000000,58.000000,7391.000000,5.000000,5.000000,1953.000000,1963.000000,0.000000
50%,2190.000000,50.000000,67.000000,9399.000000,6.000000,5.000000,1973.000000,1992.000000,0.000000
75%,2554.500000,70.000000,80.000000,11517.500000,7.000000,6.000000,2001.000000,2004.000000,164.000000
max,2919.000000,190.000000,200.000000,56600.000000,10.000000,9.000000,2010.000000,2010.000000,1290.000000


In [20]:
test_df.describe().iloc[:,10:19]

,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath
count,1458.000000,1458.000000,1459.000000,1459.000000,1459.000000,1459.000000,1457.000000,1457.000000,1459.000000
mean,554.294925,1046.117970,1156.534613,325.967786,3.543523,1486.045922,0.434454,0.065202,1.570939
std,437.260486,442.898624,398.165820,420.610226,44.043251,485.566099,0.530648,0.252468,0.555190
min,0.000000,0.000000,407.000000,0.000000,0.000000,407.000000,0.000000,0.000000,0.000000
25%,219.250000,784.000000,873.500000,0.000000,0.000000,1117.500000,0.000000,0.000000,1.000000
50%,460.000000,988.000000,1079.000000,0.000000,0.000000,1432.000000,0.000000,0.000000,2.000000
75%,797.750000,1305.000000,1382.500000,676.000000,0.000000,1721.000000,1.000000,0.000000,2.000000
max,2140.000000,5095.000000,5095.000000,1862.000000,1064.000000,5095.000000,3.000000,2.000000,4.000000


In [21]:
test_df.describe().iloc[:,20:29]

,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF
count,1459.000000,1459.000000,1459.000000,1459.00000,1459.000000,1458.000000,1458.000000,1459.000000,1459.000000
mean,2.854010,1.042495,6.385195,0.58122,1973.566141,1.766118,472.768861,93.174777,48.313914
std,0.829788,0.208472,1.508895,0.64742,31.098480,0.775945,217.048611,127.744882,68.883364
min,0.000000,0.000000,3.000000,0.00000,1895.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,1.000000,5.000000,0.00000,1956.000000,1.000000,318.000000,0.000000,0.000000
50%,3.000000,1.000000,6.000000,0.00000,1977.000000,2.000000,480.000000,0.000000,28.000000
75%,3.000000,1.000000,7.000000,1.00000,2001.000000,2.000000,576.000000,168.000000,72.000000
max,6.000000,2.000000,15.000000,4.00000,2207.000000,5.000000,1488.000000,1424.000000,742.000000


In [4]:
test_df.describe().iloc[:,30:38]

,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,1.794380,17.064428,1.744345,58.167923,6.104181,2007.769705
std,20.207842,56.609763,30.491646,630.806978,2.722432,1.301740
min,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000
50%,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,360.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000


In [47]:
test_df.iloc[:,0:27].isnull().sum() 

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage        0
LotArea            0
Street             0
Alley              0
LotShape           0
LandContour        0
LotConfig          0
LandSlope          0
Neighborhood       0
Condition1         0
Condition2         0
BldgType           0
HouseStyle         0
OverallQual        0
OverallCond        0
YearLastRemodel    0
RoofStyle          0
RoofMatl           0
Exterior1st        0
Exterior2nd        0
MasVnrType         0
MasVnrArea         0
ExterQual          0
ExterCond          0
dtype: int64

In [38]:
test_df.iloc[:,28:52].isnull().sum()

BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinSF1      0
BsmtFinType2    0
BsmtFinSF2      0
BsmtUnfSF       0
TotalBsmtSF     0
Heating         0
HeatingQC       0
CentralAir      0
Electrical      0
1stFlrSF        0
2ndFlrSF        0
LowQualFinSF    0
GrLivArea       0
BsmtFullBath    0
BsmtHalfBath    0
FullBath        0
HalfBath        0
BedroomAbvGr    0
KitchenAbvGr    0
KitchenQual     0
dtype: int64

In [37]:
test_df.iloc[:,53:80].isnull().sum() 

Functional       0
Fireplaces       0
FireplaceQu      0
GarageType       0
GarageYrBlt      0
GarageFinish     0
GarageCars       0
GarageArea       0
GarageQual       0
GarageCond       0
PavedDrive       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
PoolQC           0
Fence            0
MiscFeature      0
MiscVal          0
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
dtype: int64

#### Preprocessing train and test set

-   Fill NaN value
-   Label encoder (hashing data)
-   Combine 'YearBuilt' and 'YearRemodAdd', 'BsmtFullBath' and 'BsmtHalfBath', 'GrLivArea' and 'TotalBsmtSF'
-   Drop useless attributes

In [134]:
#Fill null value for train set
train_df['LotFrontage'].fillna(train_df['LotFrontage'].median(), inplace= True)
train_df['Alley'].fillna("None", inplace=True)
train_df['MasVnrType'].fillna("None", inplace=True)
train_df['MasVnrArea'].fillna(0, inplace=True)
train_df['BsmtQual'].fillna("None", inplace=True)
train_df['BsmtCond'].fillna("None", inplace=True)
train_df['BsmtExposure'].fillna("None", inplace=True)
train_df['BsmtFinType1'].fillna("None", inplace=True)
train_df['BsmtFinType2'].fillna("None", inplace=True)
train_df['Electrical'].fillna("Mix", inplace=True)
train_df['FireplaceQu'].fillna("None", inplace=True)
train_df['GarageType'].fillna("None", inplace=True)
train_df['GarageYrBlt'].fillna(train_df['GarageYrBlt'].min(), inplace=True)
train_df['GarageFinish'].fillna("None", inplace=True)
train_df['GarageQual'].fillna("None", inplace=True)
train_df['GarageCond'].fillna("None", inplace=True)
train_df['PoolQC'].fillna("None", inplace=True)
train_df['Fence'].fillna("None", inplace=True)
train_df['MiscFeature'].fillna("None", inplace=True)


#Fill null value for test set
test_df['LotFrontage'].fillna(train_df['LotFrontage'].median(), inplace= True)
test_df['KitchenQual'].fillna("TA", inplace=True)
test_df['Alley'].fillna("None", inplace=True)
test_df['MasVnrType'].fillna("None", inplace=True)
test_df['MasVnrArea'].fillna(0, inplace=True)
test_df['BsmtQual'].fillna("None", inplace=True)
test_df['BsmtCond'].fillna("None", inplace=True)
test_df['BsmtExposure'].fillna("None", inplace=True)
test_df['BsmtFinType1'].fillna("None", inplace=True)
test_df['BsmtFinType2'].fillna("None", inplace=True)
test_df['Electrical'].fillna("Mix", inplace=True)
test_df['FireplaceQu'].fillna("None", inplace=True)
test_df['GarageType'].fillna("None", inplace=True)
test_df['GarageYrBlt'].fillna(train_df['GarageYrBlt'].min(), inplace=True)
test_df['GarageFinish'].fillna("None", inplace=True)
test_df['GarageQual'].fillna("None", inplace=True)
test_df['GarageCond'].fillna("None", inplace=True)
test_df['PoolQC'].fillna("None", inplace=True)
test_df['Fence'].fillna("None", inplace=True)
test_df['MiscFeature'].fillna("None", inplace=True)
test_df['MSZoning'].fillna("RL", inplace=True)
test_df['Exterior1st'].fillna("VinylSd", inplace=True)
test_df['Exterior2nd'].fillna("VinylSd", inplace=True)
test_df['BsmtFinSF1'].fillna(train_df['BsmtFinSF1'].mean(), inplace=True)
test_df['BsmtFinSF2'].fillna(train_df['BsmtFinSF2'].mean(), inplace=True)
test_df['BsmtUnfSF'].fillna(train_df['BsmtUnfSF'].mean(), inplace=True)
test_df['TotalBsmtSF'].fillna(train_df['TotalBsmtSF'].mean(), inplace=True)
test_df['BsmtFullBath'].fillna(1.0, inplace=True)
test_df['BsmtHalfBath'].fillna(0, inplace=True)
test_df['Functional'].fillna("Typ", inplace=True)
test_df['GarageCars'].fillna(0, inplace=True)
test_df['GarageArea'].fillna(0, inplace=True)
test_df['SaleType'].fillna("Oth", inplace=True)



#Encode label for train set and test set
columns_to_encode = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope',
                     'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
                     'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
                     'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir',
                     'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
                     'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']

for column in columns_to_encode:
    label_encoder = LabelEncoder()
    label_encoder.fit(pd.concat([train_df[column], test_df[column]]))
    train_df[column] = label_encoder.transform(train_df[column])
    test_df[column] = label_encoder.transform(test_df[column])


#Combine 'YearBuilt' and 'YearRemodAdd', 'BsmtFullBath' and 'BsmtHalfBath', 'GrLivArea' and 'TotalBsmtSF' of train set and test set
# train_df['YearRemodAdd'] = 2023 - train_df['YearRemodAdd']
train_df = train_df.rename(columns={'YearRemodAdd': "YearLastRemodel"})

# test_df['YearRemodAdd'] = 2023 - test_df['YearRemodAdd']
test_df = test_df.rename(columns={'YearRemodAdd': "YearLastRemodel"})


train_df['BsmtFullBath'] = train_df["BsmtFullBath"] + (train_df["BsmtHalfBath"]*0.5) + train_df["FullBath"] + (train_df["HalfBath"]*0.5)
train_df = train_df.rename(columns={'BsmtFullBath': "NumberBathroom"})

test_df['BsmtFullBath'] = test_df["BsmtFullBath"] + (test_df["BsmtHalfBath"]*0.5) + test_df["FullBath"] + (test_df["HalfBath"]*0.5)
test_df = test_df.rename(columns={'BsmtFullBath': "NumberBathroom"})


train_df['TotalBsmtSF'] = train_df["GrLivArea"] + train_df["TotalBsmtSF"]
train_df = train_df.rename(columns={'TotalBsmtSF': "TotalArea"})

test_df['TotalBsmtSF'] = test_df["GrLivArea"] + test_df["TotalBsmtSF"]
test_df = test_df.rename(columns={'TotalBsmtSF': "TotalArea"})



#Scale data for train set and test set
columns_to_stdscale = ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalArea', 'MiscVal', 'LotArea', 'MasVnrArea', '1stFlrSF', '2ndFlrSF',
                    'LowQualFinSF', 'GarageArea', 'EnclosedPorch', 'MiscVal']

for column in columns_to_stdscale:
    scale = StandardScaler()
    scale.fit(train_df[[column]])
    train_df[column] = scale.transform(train_df[[column]])
    test_df[column] = scale.transform(test_df[[column]])


columns_to_mmscale = ['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea']

for column in columns_to_mmscale:
    scale = MinMaxScaler()
    scale.fit(train_df[[column]])
    train_df[column] = scale.transform(train_df[[column]])
    test_df[column] = scale.transform(test_df[[column]])

# train_df['GarageYrBlt'] = train_df['GarageYrBlt'] - 1895
# train_df['YrSold'] = train_df['YrSold'] - 2000

# test_df['GarageYrBlt'] = test_df['GarageYrBlt'] - 1895
# test_df['YrSold'] = test_df['YrSold'] - 2000


#Get train target and test Id
train_target = train_df['SalePrice']
Id = test_df['Id']

#Drop useless columns
train_df = train_df.drop(columns=['YearBuilt', 'Utilities', 'Id', 'SalePrice', 'GrLivArea', 'FullBath', 'BsmtHalfBath', 'HalfBath'])
test_df = test_df.drop(columns=['YearBuilt', 'Utilities', 'Id', 'GrLivArea', 'FullBath', 'BsmtHalfBath', 'HalfBath'])

### Initialize, train model and predict

#### XGBoost

Finetune

In [7]:
# grid search
model = XGBRegressor()
n_estimators = [140, 150, 160, 120, 130]
max_depth = [3, 4, 5, 6]
learning_rate = [0.11, 0.12, 0.13, 0.14, 0.09]
param_grid = dict(max_depth=max_depth, n_estimators=n_estimators, learning_rate = learning_rate)
grid_search = GridSearchCV(model, param_grid, cv=2, scoring = 'neg_mean_squared_error', verbose=1)
grid_result = grid_search.fit(train_df,train_target)

# summarize results
print("Best result using: %s" % (grid_result.best_params_))

Fitting 2 folds for each of 100 candidates, totalling 200 fits
Best result using: {'learning_rate': 0.12, 'max_depth': 5, 'n_estimators': 140}


Predict

In [65]:
result = pd.read_csv("data/sample_submission.csv")
result = result.drop(columns=['Id'])

In [135]:
model = XGBRegressor(max_depth=3, n_estimators=140, learning_rate = 0.12)
model.fit(train_df,train_target)

predict_test = model.predict(test_df)

print('Train error: ', np.sqrt(mean_squared_error(train_target, model.predict(train_df))), 
      'Test error: ', np.sqrt(mean_squared_error(result, model.predict(test_df))))


output = pd.DataFrame({'Id': Id, 'SalePrice': predict_test})
output.to_csv('submission.csv', index=False)

Train error:  13015.053841809864 Test error:  21786.12102753586


In [33]:
model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmsle', feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=8,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

#### RandomForest

Finetune

In [138]:
# grid search
model = RandomForestRegressor()
n_estimators = [140, 150, 160, 120, 130]
max_depth = [3, 4, 5, 6]
param_grid = dict(max_depth=max_depth, n_estimators=n_estimators)
grid_search = GridSearchCV(model, param_grid, cv=2, verbose=1)
grid_result = grid_search.fit(train_df,train_target)

# summarize results
print("Best using: %s" % (grid_result.best_params_))

Fitting 2 folds for each of 20 candidates, totalling 40 fits
Best using: {'max_depth': 6, 'n_estimators': 120}


Predict

In [122]:
model = RandomForestRegressor()
model.fit(train_df,train_target)

predict_test = model.predict(test_df)

print('Train error: ', np.sqrt(mean_squared_error(train_target, model.predict(train_df))), 
      'Test error: ', np.sqrt(mean_squared_error(result, model.predict(test_df))))

output = pd.DataFrame({'Id': Id, 'SalePrice': predict_test})
output.to_csv('submission.csv', index=False)

Train error:  10651.182254261048 Test error:  23916.68277632119
